<a href="https://colab.research.google.com/github/argosmaia/ArritmiaDataScience/blob/main/DS_P2_Argos_Maia_Paulo_Jose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Argos Maia - Paulo José**

# **As questões respondidas estarão no final desse notebook**

**Bibliotecas necessárias para rodar o programa**

In [90]:
from scipy.io import arff
import pandas as pd
import numpy as np
import warnings
import statistics as sts
import seaborn as sns
import platform
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE, SelectFromModel, VarianceThreshold, chi2
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, KBinsDiscretizer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import NeighborhoodComponentsAnalysis as nca

Inicializa o modulo base necessário para se vizualizar os dados

Foi mudado para que o programa rode em um SO e dentro de um ambiente Jupyter

In [7]:
# Verifica se está executando no Google Colab
if 'google.colab' in str(get_ipython()):
    caminho = "/content/sample_data/arritmias_treino.arff"
else:
    sistema_operacional = platform.system()

    # Verifica o sistema operacional
    if sistema_operacional == "Windows":
        caminho = r"C:\Users\seu_usuario\Área de Trabalho\Trabalho\arritmias_treino.arff"
    elif sistema_operacional == "Linux":
        caminho = "/home/seu_usuario/Área de Trabalho/Trabalho/arritmias_treino.arff"
    elif sistema_operacional == "Darwin":
        caminho = "/Users/seu_usuario/Área de Trabalho/Trabalho/arritmias_treino.arff"
    else:
        print("Sistema operacional não suportado.")

# Carrega os dados do arquivo
dados, meta = arff.loadarff(caminho)
base = pd.DataFrame(dados)
base

,Age,Sex,Height,Weight,QRS_duration,P-R,Q-T,T_interval,P_interval,QRS,...,Amp_V6_2,Amp_V6_3,Amp_V6_4,Amp_V6_5,Amp_V6_6,Amp_V6_7,Amp_V6_8,Amp_V6_9,Amp_V6_10,class
0,55.0,b'0',175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,b'1'
1,40.0,b'1',160.0,52.0,77.0,129.0,377.0,133.0,77.0,77.0,...,0.0,6.5,0.0,0.0,0.0,0.4,1.0,14.3,20.5,b'1'
2,49.0,b'1',162.0,54.0,78.0,0.0,376.0,157.0,70.0,67.0,...,0.0,8.2,-1.9,0.0,0.0,0.1,0.5,15.8,19.8,b'1'
3,44.0,b'0',168.0,56.0,84.0,118.0,354.0,160.0,63.0,61.0,...,0.0,7.0,-1.3,0.0,0.0,0.6,2.1,12.5,30.9,b'1'
4,45.0,b'1',165.0,86.0,77.0,143.0,373.0,150.0,65.0,12.0,...,0.0,4.4,-2.2,0.0,0.0,0.5,1.5,4.9,17.2,b'1'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,65.0,b'0',172.0,66.0,98.0,199.0,339.0,149.0,181.0,-4.0,...,0.0,15.0,-4.5,0.0,0.0,0.2,-0.4,26.4,23.0,b'10'
398,50.0,b'1',163.0,65.0,75.0,151.0,362.0,137.0,93.0,43.0,...,-0.4,5.1,-1.3,0.0,0.0,0.7,1.0,6.2,11.6,b'1'
399,27.0,b'1',160.0,42.0,78.0,153.0,374.0,165.0,85.0,23.0,...,-0.7,12.2,-2.4,0.0,0.0,0.5,1.9,11.0,25.0,b'1'
400,58.0,b'1',160.0,90.0,82.0,0.0,284.0,196.0,0.0,56.0,...,0.0,14.5,-2.3,0.0,0.0,0.2,-1.7,30.2,16.0,b'15'


Descrição de alguns valores da base

In [8]:
base.describe()

,Age,Height,Weight,QRS_duration,P-R,Q-T,T_interval,P_interval,QRS,T,...,Amp_V6_1,Amp_V6_2,Amp_V6_3,Amp_V6_4,Amp_V6_5,Amp_V6_6,Amp_V6_7,Amp_V6_8,Amp_V6_9,Amp_V6_10
count,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,402.000000,394.000000,...,402.000000,402.000000,402.000000,402.000000,402.000000,402.0,402.000000,402.000000,402.000000,402.000000
mean,46.883085,166.343284,68.475124,88.721393,155.579602,366.564677,169.054726,90.238806,34.927861,34.565990,...,-0.321393,-0.288308,8.956468,-1.341791,0.004478,0.0,0.506716,1.148259,19.600498,28.925622
std,16.213776,39.136777,16.603206,15.312603,46.368091,34.153636,34.178306,26.689359,43.574253,59.526263,...,0.621349,0.551296,3.428765,1.517691,0.053130,0.0,0.352164,1.416437,13.411903,17.447615
min,0.000000,105.000000,6.000000,61.000000,0.000000,232.000000,108.000000,0.000000,-172.000000,-177.000000,...,-5.600000,-4.100000,0.000000,-15.000000,0.000000,0.0,-0.800000,-6.000000,-36.600000,-38.600000
25%,36.000000,160.000000,60.000000,80.000000,141.000000,349.000000,147.250000,79.000000,8.000000,14.000000,...,-0.600000,-0.500000,6.525000,-2.100000,0.000000,0.0,0.400000,0.500000,11.350000,17.325000
50%,47.000000,164.000000,68.000000,86.000000,157.500000,367.000000,161.000000,91.000000,40.000000,41.000000,...,-0.200000,0.000000,8.700000,-1.100000,0.000000,0.0,0.500000,1.300000,18.100000,27.450000
75%,59.000000,170.000000,79.000000,94.000000,175.750000,383.000000,178.000000,102.750000,66.000000,63.000000,...,0.000000,0.000000,11.100000,0.000000,0.000000,0.0,0.700000,2.100000,26.175000,40.475000
max,83.000000,780.000000,176.000000,188.000000,524.000000,509.000000,327.000000,205.000000,169.000000,179.000000,...,2.700000,0.000000,23.600000,0.000000,0.800000,0.0,2.400000,6.000000,88.800000,97.600000


Checa-se quais bases tem valores nulos

In [9]:
base.isnull().values.any()
#checa onde se há colunas com valores vazios no dataframe
vazios = base.columns[base.isnull().any()].tolist()
print(vazios)

['T', 'P', 'QRST', 'J']


Portanto, aplicaremos a limpeza desses dados

# **Visualização de outliers, missing values, discretização e seleção de variáveis**

In [73]:
outliers = [40, 121, 296]
for outlier in outliers:
    print(base.loc[outlier, :],"\n")

Age               1.0
Sex              b'0'
Height          100.0
Weight           10.0
QRS_duration     80.0
                ...  
Amp_V6_7          0.8
Amp_V6_8          0.9
Amp_V6_9         -1.8
Amp_V6_10         5.2
class            b'5'
Name: 40, Length: 279, dtype: object 

Age               1.0
Sex              b'1'
Height          100.0
Weight            6.0
QRS_duration     85.0
                ...  
Amp_V6_7          1.3
Amp_V6_8          0.7
Amp_V6_9          2.7
Amp_V6_10         5.5
class            b'5'
Name: 121, Length: 279, dtype: object 

Age               0.0
Sex              b'0'
Height          100.0
Weight           10.0
QRS_duration     83.0
                ...  
Amp_V6_7          0.5
Amp_V6_8          2.5
Amp_V6_9        -11.8
Amp_V6_10         1.7
class            b'5'
Name: 296, Length: 279, dtype: object 



In [24]:
# Alterar a altura nas posições específicas
base.at[40, 'Height'] = 100.0
base.at[121, 'Height'] = 100.0
base.at[296, 'Height'] = 100.0

# Verificar as alterações
print(base.loc[40])
print(base.loc[121])
print(base.loc[296])

Age               1.0
Sex              b'0'
Height          100.0
Weight           10.0
QRS_duration     80.0
                ...  
Amp_V6_7          0.8
Amp_V6_8          0.9
Amp_V6_9         -1.8
Amp_V6_10         5.2
class            b'5'
Name: 40, Length: 280, dtype: object
Age               1.0
Sex              b'1'
Height          100.0
Weight            6.0
QRS_duration     85.0
                ...  
Amp_V6_7          1.3
Amp_V6_8          0.7
Amp_V6_9          2.7
Amp_V6_10         5.5
class            b'5'
Name: 121, Length: 280, dtype: object
Age               0.0
Sex              b'0'
Height          100.0
Weight           10.0
QRS_duration     83.0
                ...  
Amp_V6_7          0.5
Amp_V6_8          2.5
Amp_V6_9        -11.8
Amp_V6_10         1.7
class            b'5'
Name: 296, Length: 280, dtype: object


In [26]:
def analise_missing(base):
    # Número de NaN em cada coluna
    na_count = base.isna().sum()

    # Porcentagem de valores faltantes em cada coluna
    na_percentage = na_count / len(base) * 100

    # DataFrame com as informações sobre valores faltantes
    missing_data = pd.DataFrame({'Número de NaN': na_count, 'Porcentagem de faltantes': na_percentage})
    missing_data.sort_values(by='Número de NaN', ascending=False, inplace=True)

    # Exibir o resultado
    print(missing_data)

# Chamando a função para analisar os valores faltantes na base
analise_missing(base)

           Número de NaN  Porcentagem de faltantes
J                    334                 83.084577
P                     22                  5.472637
T                      8                  1.990050
QRST                   1                  0.248756
Amp_AVR_4              0                  0.000000
...                  ...                       ...
V2_3                   0                  0.000000
V2_4                   0                  0.000000
V2_5                   0                  0.000000
V2_6                   0                  0.000000
class                  0                  0.000000

[280 rows x 2 columns]


Apagaremos a coluna J pois ela é vazia e não tem utilidades para o programa

In [29]:
base = base.drop("J", axis=1)

In [46]:
# Get the number of missing values for each column
missing_values = base.isna().sum()

# Get the frequency of each value in each column
frequencia = base.value_counts()

# Replace the missing values with the frequency of each value
base.fillna(frequencia, inplace=True)

base

,Age,Sex,Height,Weight,QRS_duration,P-R,Q-T,T_interval,P_interval,QRS,...,Amp_V6_2,Amp_V6_3,Amp_V6_4,Amp_V6_5,Amp_V6_6,Amp_V6_7,Amp_V6_8,Amp_V6_9,Amp_V6_10,class
0,55.0,b'0',175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,b'1'
1,40.0,b'1',160.0,52.0,77.0,129.0,377.0,133.0,77.0,77.0,...,0.0,6.5,0.0,0.0,0.0,0.4,1.0,14.3,20.5,b'1'
2,49.0,b'1',162.0,54.0,78.0,0.0,376.0,157.0,70.0,67.0,...,0.0,8.2,-1.9,0.0,0.0,0.1,0.5,15.8,19.8,b'1'
3,44.0,b'0',168.0,56.0,84.0,118.0,354.0,160.0,63.0,61.0,...,0.0,7.0,-1.3,0.0,0.0,0.6,2.1,12.5,30.9,b'1'
4,45.0,b'1',165.0,86.0,77.0,143.0,373.0,150.0,65.0,12.0,...,0.0,4.4,-2.2,0.0,0.0,0.5,1.5,4.9,17.2,b'1'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,65.0,b'0',172.0,66.0,98.0,199.0,339.0,149.0,181.0,-4.0,...,0.0,15.0,-4.5,0.0,0.0,0.2,-0.4,26.4,23.0,b'10'
398,50.0,b'1',163.0,65.0,75.0,151.0,362.0,137.0,93.0,43.0,...,-0.4,5.1,-1.3,0.0,0.0,0.7,1.0,6.2,11.6,b'1'
399,27.0,b'1',160.0,42.0,78.0,153.0,374.0,165.0,85.0,23.0,...,-0.7,12.2,-2.4,0.0,0.0,0.5,1.9,11.0,25.0,b'1'
400,58.0,b'1',160.0,90.0,82.0,0.0,284.0,196.0,0.0,56.0,...,0.0,14.5,-2.3,0.0,0.0,0.2,-1.7,30.2,16.0,b'15'


In [56]:
# Verificar valores únicos nas colunas
unique_values_P = base['P'].unique()
unique_values_T = base['T'].unique()
unique_values_QRST = base['QRST'].unique()

# Calcular frequência de cada elemento nas colunas
frequency_P = base['P'].value_counts()
frequency_T = base['T'].value_counts()
frequency_QRST = base['QRST'].value_counts()

# Preencher valores faltantes com a frequência de cada elemento
base['P'].fillna(frequency_P, inplace=True)
base['T'].fillna(frequency_T, inplace=True)
base['QRST'].fillna(frequency_QRST, inplace=True)


In [58]:
# Verificar valores faltantes nas colunas 'P', 'T' e 'QRST'
missing_values_P = base['P'].isna().sum()
missing_values_T = base['T'].isna().sum()
missing_values_QRST = base['QRST'].isna().sum()

# Exibir a quantidade de valores faltantes
print("Valores faltantes na coluna 'P':", missing_values_P)
print("Valores faltantes na coluna 'T':", missing_values_T)
print("Valores faltantes na coluna 'QRST':", missing_values_QRST)


Valores faltantes na coluna 'P': 20
Valores faltantes na coluna 'T': 6
Valores faltantes na coluna 'QRST': 1


In [59]:
for i, valor in base["P"].items():
    if pd.isnull(valor):
        print(f"NaN encontrado na posição {i}")

NaN encontrado na posição 86
NaN encontrado na posição 88
NaN encontrado na posição 96
NaN encontrado na posição 113
NaN encontrado na posição 154
NaN encontrado na posição 157
NaN encontrado na posição 173
NaN encontrado na posição 184
NaN encontrado na posição 197
NaN encontrado na posição 199
NaN encontrado na posição 223
NaN encontrado na posição 233
NaN encontrado na posição 259
NaN encontrado na posição 264
NaN encontrado na posição 278
NaN encontrado na posição 280
NaN encontrado na posição 288
NaN encontrado na posição 290
NaN encontrado na posição 330
NaN encontrado na posição 400


In [61]:
#Faz a mediana de valores P
mediana_P = base["P"].dropna().median()
print(mediana_P)

55.5


In [62]:
base["P"]

0      -5.0
1      75.0
2       8.0
3      78.0
4      49.0
       ... 
397    34.0
398    49.0
399    59.0
400     NaN
401    63.0
Name: P, Length: 402, dtype: float64

In [63]:
#insere as medianas onde há valores nulos
base["P"].fillna(mediana_P, inplace=True)
base["P"]

0      -5.0
1      75.0
2       8.0
3      78.0
4      49.0
       ... 
397    34.0
398    49.0
399    59.0
400    55.5
401    63.0
Name: P, Length: 402, dtype: float64

In [74]:
for i, valor in base["QRST"].items():
    if pd.isnull(valor):
      print(f"NaN encontrado na posição {i}")

In [66]:
#Faz a mediana de valores QRST
mediana_QRST = base["QRST"].dropna().median()
print(mediana_QRST,"\n\n")
#insere as medianas onde há valores nulos
base["QRST"].fillna(mediana_QRST, inplace=True)
base["QRST"]

40.0 




0      20.0
1      65.0
2      51.0
3      66.0
4      26.0
       ... 
397    13.0
398    38.0
399    48.0
400    81.0
401    20.0
Name: QRST, Length: 402, dtype: float64

In [67]:
mediana = base["T"].dropna().median()
base["T"].fillna(mediana, inplace=True)
base["T"]

0       11.0
1       49.0
2        7.0
3       69.0
4       37.0
       ...  
397    136.0
398     33.0
399     61.0
400   -132.0
401      0.0
Name: T, Length: 402, dtype: float64

In [71]:
if 'J' in base.columns:
    if base['J'].empty:
        print("A coluna 'J' existe, mas está vazia.")
    else:
        print("A coluna 'J' existe e contém valores.")
else:
    print("A coluna 'J' não existe no DataFrame.")

A coluna 'J' não existe no DataFrame.


In [81]:
print(f"Max: {base['Height'].max()} cm")
print(f"Min: {base['Height'].min()} cm")

Max: 188.0 cm
Min: 100.0 cm


In [82]:
print(f"Max: {base['Weight'].max()}")
print(f"Min: {base['Weight'].min()}")

Max: 176.0
Min: 6.0


In [83]:
linha_weight_6 = base.loc[base['Weight'] == 6.0]
print(linha_weight_6)


     Age   Sex  Height  Weight  QRS_duration    P-R    Q-T  T_interval  \
121  1.0  b'1'   100.0     6.0          85.0  165.0  237.0       150.0   

     P_interval   QRS  ...  Amp_V6_2  Amp_V6_3  Amp_V6_4  Amp_V6_5  Amp_V6_6  \
121       106.0  88.0  ...       0.0       5.0      -4.6       0.0       0.0   

     Amp_V6_7  Amp_V6_8  Amp_V6_9  Amp_V6_10  class  
121       1.3       0.7       2.7        5.5   b'5'  

[1 rows x 279 columns]


In [84]:
# Verificar se o usuário quer imprimir apenas uma linha ou todas as linhas
print_todas = input("Deseja imprimir todas as linhas? (S/N): ").lower() == "s"

# Verificar se o usuário quer imprimir apenas uma linha
if not print_todas:
    linha_desejada = int(input("Digite o número da linha desejada: "))

# Loop para percorrer as linhas do DataFrame
for i, linha in base.iterrows():
    # Verificar se o usuário quer imprimir apenas uma linha
    if not print_todas and i != linha_desejada:
        continue  # Pular para a próxima iteração do loop

    # Imprimir os valores da idade, altura e peso
    idade = linha["Age"]
    altura = linha["Height"]
    peso = linha["Weight"]
    print(f"Linha {i + 1}: Idade={idade}, Altura={altura}, Peso={peso}")

    # Parar o loop se o usuário quiser imprimir apenas uma linha
    if not print_todas:
        break

Deseja imprimir todas as linhas? (S/N): N
Digite o número da linha desejada: 130
Linha 131: Idade=48.0, Altura=156.0, Peso=62.0


In [80]:
base['class'] = base['class'].astype(str)

# Selecionar as colunas desejadas
X = base[['T', 'P', 'QRST']].copy()
y = base['class']

# Preencher valores faltantes com a frequência de cada elemento
X['T'].fillna(X['T'].value_counts(), inplace=True)
X['P'].fillna(X['P'].value_counts(), inplace=True)
X['QRST'].fillna(X['QRST'].value_counts(), inplace=True)

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de árvore de decisão
dt_classifier = DecisionTreeClassifier()

# Treinar o modelo
dt_classifier.fit(X_train, y_train)

# Fazer previsões
previsoes = dt_classifier.predict(X_test)

# Calcular a acurácia do modelo
acuracia = accuracy_score(y_test, previsoes)
print(f"Acurácia do modelo: {acuracia:.4f}")

Acurácia do modelo: 0.3827


# **1. a) Descrição dos dados**

<p><p></p></p>

**Age**: *Numérico*(razão, continuo) = Mostra a idade dos 402 pacientes presentes na lista

**Sex**: *Categórico* (binário) - Gênero do paciente (0: Masculino, 1: Feminino) = Mostra o sexo do paciente

**Height**: *Numérico* (razão, continuo) = Mostra a altura dos pacientes

**Weight**: *Numérico* (razão, continuo) = Mostra o peso dos pacientes

**QRS_duration**: *Numérico* (razão, continuo) = Duração do complexo QRS

**P-R**: *Numérico* (razão, continuo) = intervalo P-R

**Q-T**: *Numérico* (razão, continuo) = Intervalo Q-T

**T_interval**: *Numérico* (razão, continuo) = intervalo T

**P_interval**: *Numérico* (razão, continuo) = intervalo P

**QRS**: *Numérico* (razão, continuo) = valor do complexo de QRS

**Categoria numérica (razão, continuo) de J até Amp_V6_10**: Representando valores de amplitude de canais diferentes de ESG

**Class**: Categórico (nominal, dicreta) = Classe de dados de arritmia cardíaca de 1 até 15

**Abaixo tem a códificação de cada dado**

In [ ]:
# Verificar se o usuário deseja imprimir todas as colunas ou apenas uma coluna específica
imprime_tudo = input("Deseja imprimir todas as colunas? (S/N): ").lower() == "s"

# Verificar se o usuário deseja imprimir apenas uma coluna específica
if not imprime_tudo:
    coluna_escolhida = input("Digite o nome da coluna desejada: ")

# Percorrer as colunas da base de dados
for coluna in base.columns:
    # Verificar se o usuário deseja imprimir apenas uma coluna específica
    if not imprime_tudo and coluna != coluna_escolhida:
        continue  # Pular para a próxima iteração do loop

    nome = coluna
    tipo = base[coluna].dtype

    # Verificar o tipo de dado da coluna
    if tipo == "object":
        # Atributo categórico (nominal)
        escala = "nominal"

        # Verificar a cardinalidade
        cardinalidade = "discreta" if base[coluna].nunique() <= 10 else "contínua"

    elif tipo in ["int64", "float64"]:
        # Atributo numérico (razão)
        escala = "razão"

        # Verificar a cardinalidade
        cardinalidade = "discreta" if base[coluna].nunique() <= 10 else "contínua"

        # Verificar se é uma coluna binária
        if base[coluna].nunique() == 2:
            cardinalidade = "binária"

    else:
        # Tipo de dado não reconhecido
        escala = "desconhecida"
        cardinalidade = "desconhecida"

    # Imprimir a descrição do atributo
    print("Atributo:", nome)
    print("Tipo:", tipo)
    print("Escala:", escala)
    print("Cardinalidade:", cardinalidade)
    print("---------------------------------")

Deseja imprimir todas as colunas? (S/N): S
Atributo: Age
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: Sex
Tipo: object
Escala: nominal
Cardinalidade: discreta
---------------------------------
Atributo: Height
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: Weight
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: QRS_duration
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: P-R
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: Q-T
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: T_interval
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: P_interval
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Atributo: QRS
Tipo: float64

# **Questão 1. b) Frequência dos dados**

Optamos pela codificação desses dados pois seria mais pratico para que se possa analisar o conteúdo preenchido no dataset,
ao se descrever os dados de cada atributo presente segundo sua ***frequência, max, min e dp***, analisa-se cada coluna presente

In [87]:
# Verificar se o usuário deseja imprimir todas as colunas ou apenas uma coluna específica
imprime_tudo = input("Deseja imprimir todas as colunas? (S/N): ").lower() == "s"

# Verificar se o usuário deseja imprimir apenas uma coluna específica
if not imprime_tudo:
    coluna_escolhida = input("Digite o nome da coluna desejada: ")

# Percorrer as colunas da base de dados
for coluna in base.columns:
    # Verificar se o usuário deseja imprimir apenas uma coluna específica
    if not imprime_tudo and coluna != coluna_escolhida:
        continue  # Pular para a próxima iteração do loop

    nome = coluna
    tipo = base[coluna].dtype

    # Verificar o tipo de dado da coluna
    if tipo == "object":
        # Atributo categórico (nominal)
        escala = "nominal"

        # Verificar a cardinalidade
        cardinalidade = "discreta" if base[coluna].nunique() <= 10 else "contínua"

        # Descrever a frequência dos valores
        frequencia = base[coluna].value_counts()

        # Imprimir a frequência
        print("Frequência dos valores:")
        print(frequencia)

    elif tipo in ["int64", "float64"]:
        # Atributo numérico (razão)
        escala = "razão"

        # Verificar a cardinalidade
        cardinalidade = "discreta" if base[coluna].nunique() <= 10 else "contínua"

        # Descrever estatísticas descritivas
        minimo = base[coluna].min()
        maximo = base[coluna].max()
        desvio_padrao = base[coluna].std()

        # Imprimir estatísticas descritivas
        print(f"Valor mínimo: {minimo}")
        print(f"Valor máximo: {maximo}")
        print(f"Desvio padrão: {desvio_padrao:.2f}")

    else:
        # Tipo de dado não reconhecido
        escala = "desconhecida"
        cardinalidade = "desconhecida"

    # Imprimir a descrição do atributo
    print("Atributo:", nome)
    print("Tipo:", tipo)
    print("Escala:", escala)
    print("Cardinalidade:", cardinalidade)
    print("---------------------------------")

Deseja imprimir todas as colunas? (S/N): S
Valor mínimo: 0.0
Valor máximo: 83.0
Desvio padrão: 16.21
Atributo: Age
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Frequência dos valores:
b'1'    227
b'0'    175
Name: Sex, dtype: int64
Atributo: Sex
Tipo: object
Escala: nominal
Cardinalidade: discreta
---------------------------------
Valor mínimo: 100.0
Valor máximo: 188.0
Desvio padrão: 11.06
Atributo: Height
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Valor mínimo: 6.0
Valor máximo: 176.0
Desvio padrão: 16.60
Atributo: Weight
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Valor mínimo: 61.0
Valor máximo: 188.0
Desvio padrão: 15.31
Atributo: QRS_duration
Tipo: float64
Escala: razão
Cardinalidade: contínua
---------------------------------
Valor mínimo: 0.0
Valor máximo: 524.0
Desvio padrão: 46.37
Atributo: P-R
Tipo: float64
Escala: razão
Cardinalidade: contínua
------

# **Questão 1.c)**

implementando o algoritmo J48 no dataset

Em cima, checamos se os valores discrepantes ainda existem ou se já foram substituídos

In [95]:
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Selecionar as colunas desejadas
X = base.drop('class', axis=1)
y = base['class']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo de árvore de decisão (J48)
dt_classifier = DecisionTreeClassifier(criterion='entropy')

# Treinar o modelo
dt_classifier.fit(X_train, y_train)

# Fazer previsões
previsoes = dt_classifier.predict(X_test)

# Calcular a acurácia do modelo
acuracia = accuracy_score(y_test, previsoes)
print(f"Acurácia do modelo: {acuracia:.4f}")


Acurácia do modelo: 0.6667


# **Questão 2**

Modelo   | 1º atributo | 2º atributo | N atributo
------------------------------------------------
Gain     | <p>
ReliefF  | <p>
InfoGain | <p>
CFS      | <p>

**O código abaixo vai demorar para executar**

In [106]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import accuracy_score

# Separar atributos e classe
X = base.drop('class', axis=1)
y = base['class']

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o modelo de árvore de decisão para avaliação de acurácia
dt_classifier = DecisionTreeClassifier()

# Definir os métodos de seleção de variáveis
methods = ['gain', 'relieff', 'infogain', 'cfs']

# Dicionário para armazenar os resultados de acurácia
results = {}

# Loop pelos métodos de seleção de variáveis
for method in methods:
    # Selecionar as K melhores variáveis utilizando o método específico
    if method == 'gain':
        selector = SelectKBest(score_func=mutual_info_classif, k=1)
    elif method == 'infogain':
        selector = SelectKBest(score_func=mutual_info_classif, k=1)
    elif method == 'cfs':
        selector = SelectKBest(score_func=mutual_info_classif, k=1)

    # Aplicar a seleção de variáveis no conjunto de treinamento
    X_train_selected = selector.fit_transform(X_train, y_train)

    # Obter o índice do atributo selecionado
    selected_index = selector.get_support(indices=True)[0]

    # Obter o nome do atributo selecionado
    selected_attribute = X.columns[selected_index]

    # Treinar o modelo de árvore de decisão com a variável selecionada
    dt_classifier.fit(X_train_selected, y_train)

    # Aplicar a seleção de variáveis no conjunto de teste
    X_test_selected = X_test.iloc[:, selected_index].values.reshape(-1, 1)

    # Fazer previsões com o modelo treinado
    previsoes = dt_classifier.predict(X_test_selected)

    # Calcular a acurácia para o atributo selecionado
    accuracy = accuracy_score(y_test, previsoes)

    # Armazenar o resultado de acurácia no dicionário de resultados
    results[selected_attribute] = accuracy

# Imprimir os resultados
for attribute, accuracy in results.items():
    print(f"Acurácia para o atributo {attribute}: {accuracy:.4f}")

# Identificar o atributo com maior acurácia
best_attribute = max(results, key=results.get)
print(f"Atributo com maior acurácia: {best_attribute}")


Acurácia para o atributo Amp_AVR_8: 0.5309
Acurácia para o atributo V3_3: 0.5926
Atributo com maior acurácia: V3_3
